<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/Traveling_tourist/Part%202%20-%20Exploring%20Pathfinding%20Graph%20Algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Updated to GDS 2.0 version
* Link to original blog post: https://towardsdatascience.com/part-2-exploring-pathfinding-graph-algorithms-e194ffb7f569

In [2]:
!pip install neo4j

     |████████████████████████████████| 89 kB 3.7 MB/s 
  Created wheel for neo4j: filename=neo4j-4.4.2-py3-none-any.whl size=115365 sha256=b8565e9c966a064f53e98497c6b624870c5778fb0f3deee8e6ba91866ff013c2
  Stored in directory: /root/.cache/pip/wheels/10/d6/28/95029d7f69690dbc3b93e4933197357987de34fbd44b50a0e4
Successfully built neo4j


In [3]:
# Define Neo4j connections
import pandas as pd
from neo4j import GraphDatabase
host = 'bolt://3.235.2.228:7687'
user = 'neo4j'
password = 'seats-drunks-carbon'
driver = GraphDatabase.driver(host,auth=(user, password))

def run_query(query, parameters = {}):
    with driver.session() as session:
        result = session.run(query,parameters)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

In the first part of the series, we constructed a knowledge graph of monuments located in Spain from WikiData API. Now we'll put on our graph data science goggles and explore various pathfinding algorithms available in the Neo4j Graph Data Science library. To top it off, we'll look at a brute force solution for a Santa Claus problem. Now, you might wonder what a Santa Claus problem is. It is a variation of the traveling salesman problem, except we don't require the solution to end in the same city as it started. This is because of the Santa Claus' ability to bend the time-space continuum and instantly fly back to the North Pole once he's finished with delivering goodies.
## Agenda
* Infer spatial network of monuments
* Load the in-memory projected graph with cypher projection
* Weakly connected component algorithm
* Shortest path algorithm
* Yen's k-shortest path algorithm
* Single source shortest paths algorithm
* Minimum spanning tree algorithm
* Random walk algorithm
* Traveling salesman problem
* Conclusion

### Infer spatial network of monuments
Currently, we have no direct relationships between the monuments in our graph. We do, however, have their GPS locations, which allows us to identify which monuments are nearby. This way, we can infer a spatial network of monuments.
The process is very similar to inferring a similarity network. We usually don't want to end up with a complete graph, where each node is connected to all the other ones. It would defeat the purpose of demonstrating pathfinding algorithms as the shortest path between any two nodes would always be a straight line, which would be represented as a direct relationship between the two nodes. In our case, we will connect each monument to the five closest monuments that are less than 100 kilometers away. These two numbers are entirely arbitrary. You can pick any other depending on your scenario.

In [4]:
infer_spatial_network = """

MATCH (m1:Monument),(m2:Monument) 
WHERE id(m1) > id(m2) 
WITH m1,m2, distance(m1.location_point,m2.location_point) as distance 
ORDER BY distance ASC
WHERE distance < 100000
WITH m1,collect({node:m2,distance:distance})[..5] as nearest 
UNWIND nearest as near 
WITH m1, near, near.node as nearest_node 
MERGE (m1)-[m:NEAR]-(nearest_node) SET m.distance = near.distance

"""

run_query(infer_spatial_network)

""


### Load the in-memory projected graph with cypher projection
Let's just quickly refresh how does the GDS library work.

The graph analytics pipeline consists of three parts. In the first part, the graph loader reads the stored graph from Neo4j and loads it as an in-memory projected graph. We can use either native projection or cypher projection to load the projected graph. In the second step, we execute the graph algorithms in sequence. We can use the results of one graph algorithm as an input to another. Last but not least, we store or stream the results back to Neo4j.
Here, we will use the cypher projection to load the in-memory graph. I suggest you take a look at the official documentation for more details regarding how it works. In the node statement, we will describe all monuments in our graph and add their architecture style as a node label. Adding a custom node label will allow us to filter nodes by architectural style at algorithm execution time. In the relationship statement, we will describe all the links between monuments and include the distance property, that we will use as a relationship weight.

In [5]:
load_projected_graph = """

CALL gds.graph.project.cypher('monuments',
  'MATCH (m:Monument)-[:ARCHITECTURE]->(a) 
   RETURN id(m) as id, collect(a.name) as labels',
  'MATCH (m1:Monument)-[r:NEAR]-(m2:Monument) 
   RETURN id(m1) as source, id(m2) as target, r.distance as distance')

"""

run_query(load_projected_graph)

,nodeQuery,relationshipQuery,graphName,nodeCount,relationshipCount,projectMillis
0,MATCH (m:Monument)-[:ARCHITECTURE]->(a) \n R...,MATCH (m1:Monument)-[r:NEAR]-(m2:Monument) \n ...,monuments,1299,12318,943


### Weakly connected component algorithm
Even though the weakly connected component algorithm is not a pathfinding algorithm, it is part of almost every graph analysis. It is used to find disconnected components or islands within our graph. We'll begin by running the stats mode of the algorithm.

In [6]:
wcc_stats_query = """
CALL gds.wcc.stats('monuments') 
YIELD componentCount, componentDistribution
"""

run_query(wcc_stats_query)

,componentCount,componentDistribution
0,6,"{'p99': 1270, 'min': 1, 'max': 1270, 'mean': 2..."


There are six separate components within our monuments network. The results are typical for a real-world dataset. We have a single super component that contains 98% of all nodes and a couple of tiny islands floating around. Let's examine the smaller components.

In [7]:
examine_wcc_components_query = """

CALL gds.wcc.stream('monuments')
YIELD nodeId, componentId 
WITH componentId, gds.util.asNode(nodeId) as node
OPTIONAL MATCH (node)-[:IS_IN*2..2]->(state)
RETURN componentId, 
       count(*) as component_size,
       collect(node.name) as monuments, 
       collect(distinct state.id) as state
ORDER BY component_size DESC 
SKIP 1

"""

run_query(examine_wcc_components_query)

,componentId,component_size,monuments,state
0,67,16,"[Canarias Cabrera Pinto High School, Basilica ...",[Canarias]
1,201,7,"[Arab Baths of Palma de Mallorca, Can Maneu, C...",[Illes Balears]
2,811,3,"[Church of Nuestra Señora de la Concepción, Va...",[Canarias]
3,548,2,"[San Ginés, Arrecife, Iglesia de Nuestra Señor...",[Canarias]
4,0,1,"[Fortín de Reina Regente, Melilla]",[]


hree of the five smaller components are located in the Canaries archipelago, and one is located in the Balearic Islands, specifically on Majorca. With the Neomap application, developed by Estelle Scifo, we can visualize the Canaries archipelago components on a map.

### Shortest Path algorithm
The first pathfinding graph algorithm we will use is the Shortest Path algorithm. It finds the shortest weighted path between two nodes. We define the start node and the end node and specify which relationship weight property should the algorithm take into consideration when calculating the shortest path.

In [14]:
shortest_path_query = """

MATCH (s:Monument{name:'Iglesia de Santo Domingo'}),  
      (e:Monument{name:'Colegiata de Santa María de Piasca'})
CALL gds.shortestPath.dijkstra.stream('monuments',{
  sourceNode:s,
  targetNode:e,
  relationshipWeightProperty:'distance'})
YIELD nodeIds, costs
UNWIND range(0, size(nodeIds) - 1) AS i
RETURN i, gds.util.asNode(nodeIds[i]).name AS monument, costs[i] AS cost
"""

run_query(shortest_path_query)

,i,monument,cost
0,0,Iglesia de Santo Domingo,0.000000
1,1,Iglesia de San Isidoro,363.694539
2,2,Iglesia de San Emeterio,39815.236290
3,3,Church of Santiago de Gobiendes,50922.273639
4,4,Iglesia de Santa María del Conceyu,90441.740216
5,5,"Gothic church of San Vicente, Potes",121910.069518
6,6,Colegiata de Santa María de Piasca,127242.580124


What if we wanted to plan a trip for an architectural class and visit only monuments that were influenced by either Gothic or Romanesque architecture along the way? Planning such a trip is very easy with the GDS library, as we can filter which nodes can the algorithm visit with the nodeLabels parameter.

In [15]:
shortest_path_filter_query = """

MATCH (s:Monument{name:'Iglesia de Santo Domingo'}),  
      (t:Monument{name:'Colegiata de Santa María de Piasca'})
CALL gds.shortestPath.dijkstra.stream('monuments',{
  sourceNode:s,
  targetNode:t,
  relationshipWeightProperty:'distance',
  nodeLabels:['Gothic architecture','Romanesque architecture']}) 
YIELD nodeIds, costs
UNWIND range(0, size(nodeIds) - 1) AS i
RETURN i, gds.util.asNode(nodeIds[i]).name AS monument, costs[i] AS cost

"""

run_query(shortest_path_filter_query)

,i,monument,cost
0,0,Iglesia de Santo Domingo,0.000000
1,1,Iglesia de San Juan,7133.775358
2,2,Colegiata de San Pedro de Teverga,31750.759896
3,3,Astorga Cathedral,109770.935291
4,4,"Church of San Lorenzo, Sahagún",194688.354724
5,5,Hermitage of Santa Cecilia (Vallespinoso de Ag...,265010.752425
6,6,"Church of San Cornelio y San Cipriano, Revilla...",279283.385210
7,7,"Church of San Salvador, San Salvador de Cantamuda",297916.692629
8,8,Colegiata de Santa María de Piasca,316368.966950


The route is a bit different this time as the algorithm can only visit monuments that were influenced by Gothic or Romanesque architecture style.

### Yen's k-shortest path algorithm
We have learned how to calculate the shortest weighted path between a pair of nodes. What if we were more cautious tourists and wanted to find the top three shortest paths? Having a backup plan if something unexpected might happen along the way is always a good idea. In this scenario, we could use the Yen's k-shortest path algorithm. The syntax is almost identical to the Shortest Path algorithm, except for the added k parameter, which defines how many shortest paths we would like to find.

In [16]:
yens_shortest_path_query = """

MATCH (s:Monument{name:'Iglesia de Santo Domingo'}),
      (t:Monument{name:'Colegiata de Santa María de Piasca'})
CALL gds.shortestPath.yens.stream('monuments',{
  sourceNode:s,
  targetNode:t,
  k:3,
  relationshipWeightProperty:'distance'}) 
YIELD index,nodeIds,costs
RETURN index,[nodeId in nodeIds | gds.util.asNode(nodeId).name] as monuments,apoc.coll.sum(costs) as total_cost

"""

run_query(yens_shortest_path_query)

,index,monuments,total_cost
0,0,"[Iglesia de Santo Domingo, Iglesia de San Isid...",430695.594326
1,1,"[Iglesia de Santo Domingo, Church of San Tirso...",432845.257110
2,2,"[Iglesia de Santo Domingo, Palacio de los Cond...",433141.177100


The three paths are almost the same length, just a couple hundred meters of difference. If you look closely, only the second stop is different among the three variants. Such a small difference can be attributed to the nature of our spatial network and the example pair of nodes.
### Single source shortest path algorithm
With the Single Source Shortest Path algorithm, we define the start node and search for the shortest weighted path to all the other nodes in the network. We'll inspect one of the Canaries components to limit the number of shortest paths to a reasonable number.
We'll examine the Tenerife - Gran Canaria component and choose the Cathedral of La Laguna as the starting node. The algorithm tries to find the shortest paths to all the other nodes in the network, and if no such way exists, it returns Infinity value as a result. We will filter out the unreachable nodes with the gds.util.isFinite procedure.

In [23]:
sssp_query = """

MATCH (start:Monument{name:'Cathedral of La Laguna'})
CALL gds.allShortestPaths.dijkstra.stream('monuments',
 {sourceNode:start, relationshipWeightProperty:'distance'})
YIELD index,nodeIds,costs
RETURN index,gds.util.asNode(nodeIds[-1]).name as monuments,costs[-1] as totalCost
ORDER BY totalCost ASC
"""

run_query(sssp_query)

,index,monuments,totalCost
0,0,Cathedral of La Laguna,0.000000
1,1,Canarias Cabrera Pinto High School,235.611277
2,2,Iglesia de la Concepción,420.110622
3,3,Iglesia de San Francisco de Asís,6953.377218
4,4,Iglesia de la Concepción,7141.925011
5,5,Basilica of Candelaria,16192.932014
6,6,Immaculate Conception Church,23252.693912
7,7,Shrine of Our Lady of Mount Carmel,28826.040899
8,8,"Church of San Juan Bautista, Arico",40479.026879
9,9,"Church of Santiago de los Caballeros, Galdar",75266.654817


The closest monument to the Cathedral of La Laguna is the Iglesia de la Concepción, which is just 420 meters away. It seems that there are two Iglesia de la Concepción on Tenerife Island as we can observe that it shows up twice in our results. The farthest reachable monument in our network from the Cathedral of La Laguna is Basilica of San Juan Bautista.
If we wanted to find the cost of the shortest path to all the reachable neoclassical monuments from the Cathedral of La Laguna, we could effortlessly achieve this with the nodeLabels parameter.

In [27]:
sssp_filter_query = """

MATCH (start:Monument{name:'Cathedral of La Laguna'})
CALL gds.allShortestPaths.dijkstra.stream('monuments',
 {sourceNode:start, relationshipWeightProperty:'distance',   
  nodeLabels:['Neoclassical architecture']})
YIELD index,nodeIds,costs
RETURN index,gds.util.asNode(nodeIds[-1]).name as monuments,costs[-1] as totalCost
ORDER BY totalCost ASC

"""

run_query(sssp_filter_query)

,index,monuments,totalCost
0,0,Cathedral of La Laguna,0.000000
1,1,Basilica of Candelaria,16192.932014
2,2,Basílica de Nuestra Señora del Pino,103134.802997
3,3,Mother church of Santa María de Guía,113601.351155
4,4,Cathedral of Santa Ana,113927.864686
5,5,"Church of Santiago de los Caballeros, Galdar",117417.110559


It seems there are only four neoclassical monuments on Tenerife and Gran Canaria islands.
### Minimum weight spanning tree algorithm
The Minimum Weight Spanning Tree algorithm starts from a given node and calculates a spanning tree connecting all reachable nodes with the minimum possible sum of relationship weights. For example, if we wanted to connect all the monuments in Tenerife and Gran Canaria with an optical or electric cable, we would use the Minimum Weight Spanning Tree algorithm.

In [28]:
mwst_query = """

MATCH (start:Monument{name:'Cathedral of La Laguna'})
CALL gds.alpha.spanningTree.minimum.write('monuments',{
 startNodeId:id(start),
 relationshipWeightProperty:'distance',
 weightWriteProperty:'cost'})
YIELD effectiveNodeCount
RETURN effectiveNodeCount

"""

run_query(mwst_query)

,effectiveNodeCount
0,16


Currently, only the write mode of the algorithm is available. We can visualize our potential cable route with Neomap.
### Random walk algorithm
We can imagine the Random Walk algorithm trying to mimic a drunk crowd traversing the network. They might go left, or right, take two steps forward, one step back, we never really know. It depends on how drunk the crowd is. We can use this algorithm to provide random trip recommendations. Imagine we have just visited the University of Barcelona historical building and are not sure which monuments we should take a look at next.

In [30]:
random_walk_1 = """

MATCH (m:Monument{name:"University of Barcelona historical building"})
CALL gds.beta.randomWalk.stream('monuments',
  {sourceNodes:[id(m)], walksPerNode:3, walkLength:5})
YIELD nodeIds
RETURN [nodeId in nodeIds | gds.util.asNode(nodeId).name] as result

"""

run_query(random_walk_1)

,result
0,"[University of Barcelona historical building, ..."
1,"[University of Barcelona historical building, ..."
2,"[University of Barcelona historical building, ..."


Remember, we mentioned that the Random Walk algorithm tries to mimic a drunk person traversing the network. Well, an intoxicated person might visit the same monument twice and not care. For example, in the first and third suggestions, a single monument shows up twice. Luckily, we have some options to influence how the algorithm should traverse the network in the node2vec mode with the following two parameters:
>return: This parameter controls the likelihood of immediately revisiting a node in a walk. Setting it to a high value (> max(inOut, 1)) ensures that we are less likely to sample an already visited node in the following two steps.

>inOut: This parameter allows the search to differentiate between "inward" and "outward" nodes. If inOut > 1, the random walk is biased towards nodes close to node t. In contrast, if inOut < 1, the walk is more inclined to visit nodes that are further away from the node t.

The definition of the two parameters is summarized from the original Node2vec paper.
We want to recommend monuments close to our starting point, so we choose the inOut parameter to be greater than 1. And we definitely would like to avoid revisiting an already visited node during the walk, so we choose the return parameter to be greater than the inOut parameter.

In [33]:
random_walk_2 = """

MATCH (m:Monument{name:"University of Barcelona historical building"})
CALL gds.beta.randomWalk.stream('monuments',
  {sourceNodes:[id(m)], walksPerNode:3, walkLength:5, 
   inOutFactor:5, returnFactor:10})
YIELD nodeIds
RETURN [nodeId in nodeIds | gds.util.asNode(nodeId).name] as result

"""

run_query(random_walk_2)

,result
0,"[University of Barcelona historical building, ..."
1,"[University of Barcelona historical building, ..."
2,"[University of Barcelona historical building, ..."


Unfortunately, the return parameter ensures that we are less likely to sample an already visited node in the following two steps. This means that we can't be sure that duplicates won't show up later during our walk.  In our example, Casa Batlló appears twice in the first suggestion. We can circumnavigate this problem by creating longer walk suggestions and filtering out duplicates before showing the results to the user. In the following query, we calculate nine steps long walks, filter out duplicates, and return only the first five results.

In [34]:
random_walk_3 = """

MATCH (m:Monument{name:"University of Barcelona historical building"})
CALL gds.beta.randomWalk.stream('monuments',
  {sourceNodes:[id(m)], walksPerNode:3, walkLength:5, 
   inOutFactor:5, returnFactor:10})
YIELD nodeIds
RETURN apoc.coll.toSet([nodeId in nodeIds | gds.util.asNode(nodeId).name])[..5] as result

"""

run_query(random_walk_3)

,result
0,"[University of Barcelona historical building, ..."
1,"[University of Barcelona historical building, ..."
2,"[University of Barcelona historical building, ..."


This way, we make sure the results never contain duplicates. Now we can visualize the results with our trip recommendation application.

### Traveling salesman problem
To top it off, we will solve the Santa Claus variation of the traveling salesman problem. As mentioned, the only difference is that we omit the requirement to end up in the same location as we started. I found the inspiration for this problem in the Gaming the Christmas Market post written by David Barton. I give all the credits to David Barton for conjuring up the solution. My contribution is to update it to work with Neo4j 4.0 and the GDS library.
Say we want to find the optimal route between this monuments:

In [43]:

selection = run_query("""MATCH (m:Monument) RETURN m.name AS s LIMIT 6""")['s'].to_list() 

We split the solution into two steps. First, we calculate the shortest path between all pairs of selected monuments with the gds.alpha.shortestPath algorithm and store the results as the SHORTEST_ROUTE_TO relationship between the given pair of nodes. We save the total cost and all the intermediate nodes along the shortest path as the properties of the SHORTEST_ROUTE_TO relationship.

In [44]:
traveling_salesman_part1 = """

WITH $selection as selection
MATCH (c:Monument)
WHERE c.name in selection
WITH collect(c) as monuments
UNWIND monuments as c1
WITH c1,
    [c in monuments where c.name > c1.name | c] as c2s,
    monuments
UNWIND c2s as c2
CALL gds.shortestPath.dijkstra.stream('monuments',{sourceNode:c1,targetNode:c2,relationshipWeightProperty:'distance'})
YIELD nodeIds, costs
WITH c1,
     c2,
     costs[-1] as totalCost,
     nodeIds as shortestHopNodeIds
MERGE (c1) -[r:SHORTEST_ROUTE_TO]- (c2)
SET r.cost = totalCost,
    r.shortestHopNodeIds = shortestHopNodeIds

"""

run_query(traveling_salesman_part1, {'selection': selection})

""


In the second step, we will use the apoc.path.expandConfig procedure. It enables us to perform variable-length path traversals with fine-grained control over the traversals. Check out the documentation for more details.
We allow the procedure to traverse only SHORTEST_ROUTE_TO relationships with the relationshipFilter parameter and visit only the selected monuments with the whitelistNodes parameter. We ensure that all selected nodes must be visited exactly once by defining the number of hops or levels traversed (minLevel and maxLevel) and with the uniqueness parameter. I know it is a lot to comprehend, and if you need some help, I would suggest asking questions on the Neo4j community site. We then select the path with the minimum sum of relationship weights as the solution. Because we calculate all the possible routes between the chosen monuments, this is a brute-force solution of the traveling salesman problem.

In [47]:
traveling_salesman_part2 = """

WITH $selection as selection
MATCH (c:Monument) 
WHERE c.name in selection
WITH collect(c) as monuments
UNWIND monuments as c1
WITH c1,
    [c in monuments where c.name > c1.name | c] as c2s,
    monuments,
    (size(monuments) - 1) as level
UNWIND c2s as c2
CALL apoc.path.expandConfig(c1, {
  relationshipFilter: 'SHORTEST_ROUTE_TO',
  minLevel: level,
  maxLevel: level,
  whitelistNodes: monuments,
  terminatorNodes: [c2],
  uniqueness: 'NODE_PATH'}) 
YIELD path
WITH path,
    reduce(cost = 0, x in relationships(path) | cost + x.cost) as totalCost
ORDER BY totalCost LIMIT 1
WITH path,
     totalCost,
     apoc.coll.flatten([r in relationships(path) | r.shortestHopNodeIds]) as intermediate_stops,
     [n in nodes(path) | id(n)] as node_ids
RETURN  [n in nodes(path) | n.name] as path,
        round(totalCost, 2) as total_distance,
        [optional in intermediate_stops where not optional in node_ids | gds.util.asNode(optional).name] as optional_stops

"""

run_query(traveling_salesman_part2, {'selection': selection})

,path,total_distance,optional_stops


In the path column of the results, we have an ordered array of selected monuments to visit. Our travel would start with Castell de Sant Jaume and continue to Castell de Vilaüt and so on. We could dub this the Spanish castle-visiting trip as we selected six castles, and we have an option to see four more along the way. The total air distance of the path is 126 kilometers.